In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))

# X = torch.rand(2,20) 这行代码会生成一个大小为 2x20 的张量，其中包含了 40 个在区间 [0, 1) 内均匀分布的随机数。
X = torch.rand(2,20)     # 输入

net(X)   # 输出

tensor([[ 0.0191,  0.0933,  0.1500,  0.0929,  0.1964, -0.0390,  0.1997, -0.1673,
          0.0278, -0.0033],
        [-0.0475,  0.0938,  0.1908,  0.0760, -0.0488,  0.1465,  0.1322, -0.1653,
          0.0159, -0.1065]], grad_fn=<AddmmBackward>)

In [3]:
class MyMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
    def forward(self, X):
        return self.out(F.relu((self.hidden(X))))      # 将三个层链接起来

net = MyMLP()
X = torch.rand(2,20)
net(X)

tensor([[ 0.1107, -0.0152, -0.0482,  0.0857, -0.0134, -0.1858,  0.1200, -0.0125,
         -0.2319,  0.2418],
        [ 0.1450, -0.0364,  0.0737,  0.0632, -0.0144, -0.2498,  0.1886,  0.0132,
         -0.2994,  0.2674]], grad_fn=<AddmmBackward>)

In [4]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for block in args:
            print(block)
            self._modules[block] = block
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

Linear(in_features=20, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)


tensor([[ 0.1465, -0.2628, -0.2069, -0.0105,  0.1629,  0.0803,  0.2052, -0.0417,
         -0.0517,  0.1107],
        [ 0.1986, -0.1893, -0.1800,  0.1601,  0.2471,  0.1711,  0.1583, -0.0239,
         -0.0810,  0.0832]], grad_fn=<AddmmBackward>)

In [5]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                 nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
    def forward(self,X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),MyMLP())
X = torch.rand(2,20)
chimera(X)

tensor([[ 0.0536,  0.0049, -0.0555, -0.0426,  0.0314, -0.0742, -0.1588, -0.0821,
          0.0488, -0.1305],
        [ 0.0572,  0.0065, -0.0543, -0.0418,  0.0279, -0.0718, -0.1583, -0.0794,
          0.0450, -0.1323]], grad_fn=<AddmmBackward>)

In [6]:
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

tensor([[0.1880],
        [0.1960]], grad_fn=<AddmmBackward>)

In [7]:
print(net[2].state_dict())
print()
print(type(net[2].bias))
print()
print(net[2].bias)
print()
print(net[2].bias.data)
print()
print(net[2].weight.grad==None)

OrderedDict([('weight', tensor([[ 0.1474, -0.0005,  0.3029,  0.3496, -0.2981, -0.2374, -0.1073,  0.3436]])), ('bias', tensor([0.2570]))])

<class 'torch.nn.parameter.Parameter'>

Parameter containing:
tensor([0.2570], requires_grad=True)

tensor([0.2570])

True


In [8]:
print(*[(name,param.shape)for name,param in net[0].named_parameters()])
print()
print(*[(name, param.shape) for name, param in net.named_parameters()])
print()
print(net.state_dict()['2.bias'].data)

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))

tensor([0.2570])


In [9]:
def block1():
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[0.0334],
        [0.0334]], grad_fn=<AddmmBackward>)

In [10]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [11]:
def init_normal(m):    # 传入的是一个module
    if type(m) == nn.Linear:     # 如果是线性层
        nn.init.normal_(m.weight, mean=0, std=0.01)     # _表示无需返回值，直接原地替换。这里将weight变成均值为0，标准差为0.01
        nn.init.zeros_(m.bias)    # 将bias偏移替换为0

net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

net.apply(init_normal)      # 将网络遍历一遍，用init_normal进行更新
net[0].weight.data[0],net[0].bias.data[0]

(tensor([-0.0038, -0.0017, -0.0045,  0.0039]), tensor(0.))

In [12]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [13]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)      # 对weight做xavier均值化

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)      # 全部赋值42

net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

net[0].apply(xavier)
net[2].apply(init_42)

print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5082, -0.0528, -0.1069,  0.1827])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [14]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name,param.shape) for name, param in m.named_parameters()][0]
        )
        nn.init.uniform_(m.weight, -10, 10)      # (-10,10)之间的均匀分布填充
        m.weight.data *= m.weight.data.abs()>=5     # 表示如果 m.weight.data.abs()>=5成立，相当于1；不成立相当于0

net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[8.3302, 0.0000, 0.0000, -0.0000],
        [9.1558, 9.6040, -0.0000, 0.0000]], grad_fn=<SliceBackward>)

In [15]:
net[0].weight.data[:] += 1     # 所有值+1
net[0].weight.data[0, 0] = 42     # 第一行的第一个元素为42
net[0].weight.data[0]      # 第一行元素

tensor([42.,  1.,  1.,  1.])

In [16]:
shared = nn.Linear(8,8)
net = nn.Sequential(
    nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1)
)
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])

net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [17]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [18]:
net = nn.Sequential(nn.Linear(8,128), CenteredLayer())

Y = net(torch.rand(4,8))
Y.mean()

tensor(7.4506e-09, grad_fn=<MeanBackward0>)

In [19]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units))      # 均值为0，方差为1的正态分布，模型形状为 in_units*units
        self.bias = nn.Parameter(torch.zeros(units))
    def forward(self,X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5,3)
dense.weight

Parameter containing:
tensor([[-1.4344,  0.5694, -0.6562],
        [ 0.8137,  0.1474, -1.5661],
        [-0.4021,  0.6279,  0.7612],
        [-0.3380, -1.6948,  0.6764],
        [-0.6898,  0.9873,  1.0886]], requires_grad=True)

In [20]:
dense(torch.rand(2,5))

tensor([[0.2325, 0.0000, 0.0000],
        [0.0000, 0.5239, 0.9386]])

In [21]:
net = nn.Sequential(MyLinear(64,8), MyLinear(8,1))
net(torch.rand(2,64))

tensor([[0.],
        [0.]])

In [22]:
x = torch.arange(4)
torch.save(x, 'x-file')      # 保存数据到文件

x2 = torch.load("x-file")    # 从文件读取数据
x2

tensor([0, 1, 2, 3])

In [23]:
y = torch.zeros(4)
torch.save([x,y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [24]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [29]:
class MyMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)
    def forward(self, X):
        return self.output(F.relu((self.hidden(X))))      # 将三个层链接起来

net = MyMLP()
X = torch.randn(size=(2,20))
Y = net(X)
print(net.state_dict())
torch.save(net.state_dict(),'mlp.params')

OrderedDict([('hidden.weight', tensor([[ 0.0971, -0.1579,  0.0044,  ..., -0.0998,  0.0737, -0.1686],
        [ 0.0530,  0.0414,  0.1216,  ...,  0.1181, -0.0419,  0.2201],
        [-0.1028, -0.1758, -0.1662,  ..., -0.0928,  0.1294,  0.0587],
        ...,
        [-0.1926, -0.1774,  0.2134,  ..., -0.2065, -0.1961, -0.0740],
        [ 0.1167,  0.1235,  0.0281,  ..., -0.2233, -0.1973,  0.2134],
        [ 0.1455,  0.0599,  0.1002,  ...,  0.0954, -0.1207, -0.1105]])), ('hidden.bias', tensor([-0.0302, -0.0348,  0.0281,  0.0997, -0.0086,  0.1300,  0.0476, -0.2219,
         0.1233,  0.0026, -0.0673,  0.0856, -0.1732,  0.1215, -0.1404, -0.1227,
         0.2165,  0.1321, -0.0280,  0.0206, -0.0785, -0.2105, -0.0730,  0.1728,
         0.0761,  0.1807,  0.1850,  0.1304,  0.1791, -0.1322, -0.1459, -0.2088,
         0.1379,  0.0745, -0.1551,  0.1028, -0.0318, -0.0190,  0.0452, -0.1210,
        -0.1974,  0.0848, -0.1211, -0.0029, -0.0758, -0.1973, -0.1604,  0.1820,
         0.1789,  0.0604,  0.2105,  0

In [30]:
clone = MyMLP()     # 先声明一个MyMLP，此时网络已经初始化
clone.load_state_dict(torch.load("mlp.params"))    # 更新权重
print(clone.eval())

Y_clone = clone(X)
print(Y_clone == Y)

MyMLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)
tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])
